# RNN text generation using LSTM

In [1]:
import torch
import numpy as np 
import torch.nn as nn
from torch.nn.utils import clip_grad_norm
import wordninja
path='/mnt/disk1/Gulshan/rnn/guitar rnn.txt'

In [2]:
class dictionary:
    def __init__(self):
        self.word2idx={}
        self.idx2word={}
        self.counter=0
    def create_word(self,word):
        if word not in self.word2idx:
            self.word2idx[word]=self.counter
            self.idx2word[self.counter]=word
            self.counter+=1
    def __len__(self):
        return len(self.word2idx)

In [3]:
class preprocess(dictionary):
    def get_data(self,path,batch_size=10):
        with open(path,'r') as f:
            token=0
            for line in f:
                words=line.split()+['<eos>']
                token+=len(words)
                for word in words:
                    self.create_word(word)
        rep_tensor = torch.LongTensor(token)
        index=0
        with open(path,'r') as f:
            for line in f:
                words=line.split()+['<eos>']
                for word in words:
                        rep_tensor[index] = self.word2idx[word]
                        index += 1
        num_batch=rep_tensor.shape[0]//batch_size
        rep_tensor=rep_tensor[:num_batch*batch_size] #Remove the remainder (Filter out the ones that don't fit) remove remaining
        rep_tensor=rep_tensor.view(batch_size,-1)
        return rep_tensor
        

In [4]:
# rep_tensor = torch.LongTensor(10)
# rep_tensor
# rep_tensor[9]=1
# b=2
# n=rep_tensor.shape[0]//b
# print(rep_tensor,rep_tensor.shape)
# rep_tensor=rep_tensor[:b*n]
# print(rep_tensor,rep_tensor.shape)
# rep_tensor=rep_tensor.view(b,-1)


In [5]:
corpus=preprocess()
rep_tensor=corpus.get_data('/mnt/disk1/Gulshan/rnn/guitar rnn.txt')

In [6]:
rep_tensor.shape

torch.Size([10, 41])

In [7]:
embed_size=120
hidden_size=1000
num_layer=1
epochs=50
batch_size=10
l_r=0.001
timestep=25
vocab_size=len(corpus.word2idx)
vocab_size

231

In [8]:
class textgenerator(nn.Module):
    def __init__(self,vocab_size,embed_size,hidden_size,num_layer):
        super(textgenerator,self).__init__()
        self.embed=nn.Embedding(vocab_size,embed_size)
        self.lstm=nn.LSTM(embed_size,hidden_size,num_layer,batch_first=True)
        self.linear=nn.Linear(hidden_size,vocab_size)
    
    def forward(self,x,h):
        x=self.embed(x) # perform word embedding
        # print(x.shape)
        out,(h,c)=self.lstm(x,h)
        # print(out.shape)
        out=out.reshape(out.size(0)*out.size(1),out.size(2)) #reshape accor.. (batch_size*timestep,hidden) for linear fc layer
        # print(out.shape)
        out=self.linear(out)
        # print(out.shape)
        
        return out,(h,c)
        

In [9]:
model=textgenerator(vocab_size,embed_size,hidden_size,num_layer)

In [10]:
# input=torch.randn(1,1).long()
# print(input.shape)
# states=(torch.zeros(num_layer,1,hidden_size),
#             torch.zeros(num_layer,1,hidden_size))
# x=model(input,states)

In [11]:
loss_fn=nn.CrossEntropyLoss()
optim=torch.optim.Adam(model.parameters(),lr=l_r)
# rep_tensor.shape

In [12]:
for epoch in range(epochs):
    states=(torch.zeros(num_layer,batch_size,hidden_size),
            torch.zeros(num_layer,batch_size,hidden_size)) # hidden state and ,memory state
    for i in range(0,rep_tensor.size(1)-timestep,timestep):
        # print(i)
        # print(timestep)
        input=rep_tensor[:,i:i+timestep] # [batch,num-of_elemments] # input - i like guitar
        target=rep_tensor[:,i+1:(i+1)+timestep] #[batch_size,i+1 element] #target - like guitar s (so)
        print(input.shape)
        # print(rep_tensor.shape)
        # print(target.shape)
        
        out,_=model(input,states)
        # print(out.shape)
        # print(target.reshape(-1).shape)
        loss=loss_fn(out,target.reshape(-1))
        
        model.zero_grad()
        loss.backward()
        
        clip_grad_norm(model.parameters(),0.5)
        
        optim.step()
        break
        step=(i+1)//timestep
        if step%100==0:
            print(f'Epoch [{epoch+1}/{epochs}] , loss {loss.item():.4f}')
    

torch.Size([10, 25])


/tmp/ipykernel_3809584/2606486995.py:21: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  clip_grad_norm(model.parameters(),0.5)


torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([10, 25])
torch.Size([1

In [13]:
# with torch.no_grad():
#     # with open('result_guitar_rnn.txt','w') as f:
#         states=(torch.zeros(num_layer,1,hidden_size), # batch_size is 1
#             torch.zeros(num_layer,1,hidden_size))
#         # random no for testing starting from 0 to  vocab size and dimension is 1 and adding batch size
#         input=torch.randint(0,vocab_size,(1,)).long().unsqueeze(1) 
#         print(input)
#         for i in range(400):
#             out,_=model(input,states)
#             print(out.shape)
#             # print(out)
#             prob=out.exp()
#             print(prob.shape)
#             word_id=torch.multinomial(prob,num_samples=1).item()
#             print(word_id)
#             input.fill_(word_id) # replace inpuit for next word
            
#             print(input)
            
#             word=corpus.idx2word[word_id]
#             print(word)
#             word='\n' if word =='<eos>' else word + ' '
#             # f.write(word)
#             if (i==2):
#                 break
#             if (i+1)%100==0:
#                 print(f'sampled [{i+1}/{400} words and save to result_gutarrnn.txt]')
              
        

In [14]:
input=torch.randint(0,vocab_size,(1,))
input=input.long()#.unsqueeze(1)
input

tensor([18])